---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [190]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

In [225]:
import numpy as np

In [221]:
d1_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
d1_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
d1 = pd.concat([d1_1,d1_2])
d1.reset_index(inplace=True)
d1_index = d1['level_0']
test = pd.DataFrame(d1)
test[test[1].apply(int) > 31 ]

,level_0,match,0,1,2


array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [228]:
def date_sorter():
    # 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    d1_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    d1_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    d1 = pd.concat([d1_1,d1_2])
    d1.reset_index(inplace=True)
    d1_index = d1['level_0']
    #print(d1_index)
    
    # Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
    d2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    d2.reset_index(inplace=True)
    d2_index = d2['level_0']
    #print(d2)
    
    # 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    # Feb 2009; Sep 2009; Oct 2010
    d3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    d3.reset_index(inplace=True)
    d3_index = d3['level_0']
    #print(d3)
    
    # Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
    #d4 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ]*(?:\d{1,2}(?:st|nd|rd|th), )\d{4})')
    #print(d4)
    
    # Feb 2009; Sep 2009; Oct 2010
    #d5 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ]*\d{4})')
    #print(d5)
    
    # 6/2008; 12/2009
    d6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    d6.reset_index(inplace=True)
    d6_index = d6['level_0']
    s = []
    for i in d6_index:
        if not(i in d1_index.values):
            s.append(i)
    #print(d6)
    s = np.asarray(s)
    d6 = d6[d6['level_0'].isin(s)]
    #print(d6)
    
    # 6/2008; 12/2009
    d7_1 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    d7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    d7 = pd.concat([d7_1,d7_2])
    d7.reset_index(inplace=True)
    d7_index = d7['level_0']
    s = []
    for i in d7_index:
        if not((i in d2_index.values) | (i in d3_index.values) | (i in d6_index.values)):
            s.append(i)
    #print(d6)
    s = np.asarray(s)
    d7 = d7[d7['level_0'].isin(s)]
    #print(d7)
    
    s = d1.level_0.values.tolist()+d2.level_0.values.tolist()+d3.level_0.values.tolist()+d6.level_0.values.tolist()+d7.level_0.values.tolist()
    #print(s)
    
    d1.columns=['level_0','match','month','day','year']
    d1['year'] = d1['year'].apply(str)
    d1['year'] = d1['year'].apply(lambda x : '19'+x if len(x)<=2 else x)
    #print(d1)
    
    d2[1] = d2[1].apply(lambda x: x.replace(',',''))
    d2['day'] = d2[1].apply(lambda x: x.split(' ')[0])
    d2['year'] = d2[1].apply(lambda x: x.split(' ')[1])
    d2.columns=['level_0','match','month','day-year','day','year']
    d2.drop('day-year',axis=1,inplace=True) 
    #print(d2)
    
    d3.columns=['level_0','match','day','month','year']
    d3['day'] = d3['day'].apply(lambda x: 1 if(pd.isnull(x)) else x)
    d3['month'] = d3['month'].apply(lambda x: x[:3])
    d3['month'] = pd.to_datetime(d3.month, format='%b').dt.month
    #print(d3)
    
    d6.columns=['level_0','match','month','year']
    d6['day'] = 1
    #print(d6)
    
    d7.columns=['level_0','match','year']
    d7['day']=1
    d7['month']=1
    #print(d7)
    
    d = pd.concat([d1,d2,d3,d6,d7])
    d['date'] = pd.to_datetime((d['month'].apply(str)+'/'+d['day'].apply(str)+'/'+d['year'].apply(str)))
    
    d_final = d.sort_values(['level_0']).set_index('level_0')
    
    print_list = d_final['date']
    #print(d_final.day.unique())
    answer = pd.Series([i[0] for i in sorted(enumerate(print_list), key=lambda x:x[1])])
    return answer
date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64